# 5.4 안전 운전자 예측 경진대회 성능 개선 I : LightGBM 모델

- [안전 운전자 예측 경진대회 링크](https://www.kaggle.com/c/porto-seguro-safe-driver-prediction)
- [모델링 노트북 참고 링크](https://www.kaggle.com/xiaozhouwang/2nd-place-lightgbm-solution)

In [1]:
# ch5/safe_driver_prediction_lgb_modeling.ipynb

import numpy as np
import pandas as pd

# 데이터 경로
data_path = '/kaggle/input/porto-seguro-safe-driver-prediction/'

train = pd.read_csv(data_path + 'train.csv', index_col='id')
test = pd.read_csv(data_path + 'test.csv', index_col='id')
submission = pd.read_csv(data_path + 'sample_submission.csv', index_col='id')

## 5.4.1 피처 엔지니어링

### 데이터 합치기

In [2]:
all_data = pd.concat([train, test], ignore_index=True)
all_data = all_data.drop('target', axis=1) # 타깃 값 제거

all_features = all_data.columns # 전체 피처

### 명목형 피처 원-핫 인코딩

In [3]:
from sklearn.preprocessing import OneHotEncoder

# 명목형 피처
cat_features = [feature for feature in all_features if 'cat' in feature] 

# 원-핫 인코딩 적용
onehot_encoder = OneHotEncoder()
encoded_cat_matrix = onehot_encoder.fit_transform(all_data[cat_features]) 

### 파생 피처 추가

In [4]:
# '데이터 하나당 결측값 개수'를 파생 변수로 추가
all_data['num_missing'] = (all_data==-1).sum(axis=1)

In [5]:
# 명목형 피처, calc 분류 피처를 제외한 피처
remaining_features = [feature for feature in all_features
                      if ('cat' not in feature and 'calc' not in feature)] 
# num_missing을 remaining_features에 추가
remaining_features.append('num_missing')

In [6]:
# 분류가 ind인 피처
ind_features = [feature for feature in all_features if 'ind' in feature]

is_first_feature=True
for ind_feature in ind_features:
    if is_first_feature:
        all_data['mix_ind'] = all_data[ind_feature].astype(str)+'_'
        is_first_feature = False
    else:
        all_data['mix_ind'] += all_data[ind_feature].astype(str)+'_'

In [7]:
all_data['mix_ind']

0          2_2_5_1_0_0_1_0_0_0_0_0_0_0_11_0_1_0_
1           1_1_7_0_0_0_0_1_0_0_0_0_0_0_3_0_0_1_
2          5_4_9_1_0_0_0_1_0_0_0_0_0_0_12_1_0_0_
3           0_1_2_0_0_1_0_0_0_0_0_0_0_0_8_1_0_0_
4           0_2_0_1_0_1_0_0_0_0_0_0_0_0_9_1_0_0_
                           ...                  
1488023     0_1_6_0_0_0_1_0_0_0_0_0_0_0_2_0_0_1_
1488024    5_3_5_1_0_0_0_1_0_0_0_0_0_0_11_1_0_0_
1488025     0_1_5_0_0_1_0_0_0_0_0_0_0_0_5_0_0_1_
1488026    6_1_5_1_0_0_0_0_1_0_0_0_0_0_13_1_0_0_
1488027    7_1_4_1_0_0_0_0_1_0_0_0_0_0_12_1_0_0_
Name: mix_ind, Length: 1488028, dtype: object

In [8]:
cat_count_features = []
for feature in cat_features+['mix_ind']:
    val_counts_dict = all_data[feature].value_counts().to_dict()
    all_data[f'{feature}_count'] = all_data[feature].apply(lambda x: 
                                                           val_counts_dict[x])
    cat_count_features.append(f'{feature}_count')

In [9]:
cat_count_features

['ps_ind_02_cat_count',
 'ps_ind_04_cat_count',
 'ps_ind_05_cat_count',
 'ps_car_01_cat_count',
 'ps_car_02_cat_count',
 'ps_car_03_cat_count',
 'ps_car_04_cat_count',
 'ps_car_05_cat_count',
 'ps_car_06_cat_count',
 'ps_car_07_cat_count',
 'ps_car_08_cat_count',
 'ps_car_09_cat_count',
 'ps_car_10_cat_count',
 'ps_car_11_cat_count',
 'mix_ind_count']

### 필요 없는 피처 제거

In [10]:
from scipy import sparse

# 필요 없는 피처들
drop_features = ['ps_ind_14', 'ps_ind_10_bin','ps_ind_11_bin', 
                 'ps_ind_12_bin','ps_ind_13_bin','ps_car_14']

# remaining_features, cat_count_features에서 drop_features를 제거한 데이터
all_data_remaining = all_data[remaining_features+cat_count_features].drop(drop_features, axis=1)

# 데이터 합치기
all_data_sprs = sparse.hstack([sparse.csr_matrix(all_data_remaining),
                               encoded_cat_matrix],
                              format='csr')

### 데이터 나누기

In [11]:
num_train = train.shape[0] # 훈련 데이터 개수

# 훈련 데이터와 테스트 데이터 나누기
X = all_data_sprs[:num_train]
X_test = all_data_sprs[num_train:]

y = train['target'].values

### 정규화 지니계수 계산 함수

In [12]:
def eval_gini(y_true, y_pred):
    # 실제 값과 예측 값의 크기가 같은지 확인 (값이 다르면 오류 발생)
    assert y_true.shape == y_pred.shape

    n_samples = y_true.shape[0] # 데이터 개수
    L_mid = np.linspace(1 / n_samples, 1, n_samples) # 대각선 값

    # 1) 예측 값에 대한 지니계수
    pred_order = y_true[y_pred.argsort()] # y_pred 크기순으로 y_true 값 정렬
    L_pred = np.cumsum(pred_order) / np.sum(pred_order) # 로렌츠 곡선
    G_pred = np.sum(L_mid - L_pred)# 예측 값에 대한 지니계수

    # 2) 예측이 완벽할 때 지니계수
    true_order = y_true[y_true.argsort()] # y_true 크기순으로 y_true 값 정렬
    L_true = np.cumsum(true_order) / np.sum(true_order) # 로렌츠 곡선
    G_true = np.sum(L_mid - L_true) # 예측이 완벽할 때 지니계수

    # 정규화된 지니계수
    return G_pred / G_true

In [13]:
def gini(preds, dtrain):
    labels = dtrain.get_label()
    return 'gini', eval_gini(labels, preds), True

## 5.4.2 하이퍼 파라미터 최적화

### 데이터 세트 준비

In [14]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split

# 8:2 비율로 훈련 데이터, 검증 데이터 분리 (베이지안 최적화 수행용)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                      test_size=0.2, 
                                                      random_state=0)

# 베이지안 최적화용 데이터 세트
bayes_dtrain = lgb.Dataset(X_train, y_train)
bayes_dvalid = lgb.Dataset(X_valid, y_valid)

### 하이퍼 파라미터 범위 설정

In [15]:
# 베이지안 최적화를 위한 하이퍼 파라미터 범위
param_bounds = {'num_leaves': (28, 33),
                'lambda_l1': (0.9, 1),
                'lambda_l2': (0.9, 1),
                'feature_fraction': (0.6, 0.8),
                'bagging_fraction': (0.7, 0.9),
                'min_child_samples': (8, 12),
                'min_child_weight': (120, 130)}

# 값이 고정된 하이퍼 파라미터
fixed_params = {'objective': 'binary',
                'learning_rate': 0.01,
                'bagging_freq': 1,
                'force_row_wise': True,
                'random_state': 1991}

### (베이지안 최적화용) 평가지표 계산 함수 작성

In [16]:
def eval_function(num_leaves, lambda_l1, lambda_l2, feature_fraction,
                  bagging_fraction, min_child_samples, min_child_weight):
    '''최적화하려는 평가지표(지니계수) 계산 함수'''
    
    # 베이지안 최적화를 수행할 하이퍼 파라미터 ---①
    params = {'num_leaves': int(round(num_leaves)),
              'lambda_l1': lambda_l1,
              'lambda_l2': lambda_l2,
              'feature_fraction': feature_fraction,
              'bagging_fraction': bagging_fraction,
              'min_child_samples': int(round(min_child_samples)),
              'min_child_weight': min_child_weight,
              'feature_pre_filter': False}
    # 고정된 하이퍼 파라미터도 추가 ---②
    params.update(fixed_params)
    
    print('하이퍼 파라미터:', params)    
    
    # LightGBM 모델 훈련
    lgb_model = lgb.train(params=params, 
                           train_set=bayes_dtrain,
                           num_boost_round=1500,
                           valid_sets=bayes_dvalid,
                           feval=gini,
                           early_stopping_rounds=150,
                           verbose_eval=False)
    
    best_iter = lgb_model.best_iteration # 최적 반복 횟수
    
    # 검증 데이터로 예측 수행
    preds = lgb_model.predict(X_valid, num_iteration=best_iter) 
    
    # 지니계수 계산
    gini_score = eval_gini(y_valid, preds)
    print(f'지니계수: {gini_score}\n')
    
    return gini_score

### 최적화 수행

In [17]:
from bayes_opt import BayesianOptimization

# 베이지안 최적화 객체 생성
optimizer = BayesianOptimization(f=eval_function, # 평가지표 계산 함수
                                 pbounds=param_bounds, # 하이퍼 파라미터 범위
                                 random_state=0)

In [18]:
# 베이지안 최적화 수행
optimizer.maximize(init_points=5, n_iter=10)

|   iter    |  target   | baggin... | featur... | lambda_l1 | lambda_l2 | min_ch... | min_ch... | num_le... |
-------------------------------------------------------------------------------------------------------------
하이퍼 파라미터: {'num_leaves': 30, 'lambda_l1': 0.9602763376071644, 'lambda_l2': 0.9544883182996897, 'feature_fraction': 0.7430378732744839, 'bagging_fraction': 0.809762700785465, 'min_child_samples': 10, 'min_child_weight': 126.45894113066656, 'feature_pre_filter': False, 'objective': 'binary', 'learning_rate': 0.01, 'bagging_freq': 1, 'force_row_wise': True, 'random_state': 1991}
[LightGBM] [Info] Number of positive: 17383, number of negative: 458786
[LightGBM] [Info] Total Bins 1555
[LightGBM] [Info] Number of data points in the train set: 476169, number of used features: 217
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.036506 -> initscore=-3.273091
[LightGBM] [Info] Start training from score -3.273091
지니계수: 0.2842759041022516

|  1        |  0.2843   |  0.8098   |  0

### 결과 확인

In [19]:
# 평가함수 점수가 최대일 때 하이퍼 파라미터
max_params = optimizer.max['params']
max_params

{'bagging_fraction': 0.8043696643500143,
 'feature_fraction': 0.6829323879981047,
 'lambda_l1': 0.9264555612104627,
 'lambda_l2': 0.9774233689434216,
 'min_child_samples': 9.824601328866194,
 'min_child_weight': 125.68433948868649,
 'num_leaves': 28.093949002181777}

In [20]:
# 정수형 하이퍼 파라미터 변환
max_params['num_leaves'] = int(round(max_params['num_leaves']))
max_params['min_child_samples'] = int(round(max_params['min_child_samples']))

In [21]:
# 값이 고정된 하이퍼 파라미터 추가
max_params.update(fixed_params)

In [22]:
max_params

{'bagging_fraction': 0.8043696643500143,
 'feature_fraction': 0.6829323879981047,
 'lambda_l1': 0.9264555612104627,
 'lambda_l2': 0.9774233689434216,
 'min_child_samples': 10,
 'min_child_weight': 125.68433948868649,
 'num_leaves': 28,
 'objective': 'binary',
 'learning_rate': 0.01,
 'bagging_freq': 1,
 'force_row_wise': True,
 'random_state': 1991}

## 5.4.3 훈련 및 예측

In [23]:
from sklearn.model_selection import StratifiedKFold

# Stratified K 폴드 교차검증기 생성
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1991)

# OOF 방식으로 훈련된 모델로 검증 데이터 타깃 값을 예측한 확률을 담을 1차원 배열
oof_val_preds = np.zeros(X.shape[0]) 
# OOF 방식으로 훈련된 모델로 테스트 데이터 타깃 값을 예측한 확률을 담을 1차원 배열
oof_test_preds = np.zeros(X_test.shape[0]) 

# OOF 방식으로 모델 훈련, 검증, 예측
for idx, (train_idx, valid_idx) in enumerate(folds.split(X, y)):
    # 각 폴드를 구분하는 문구
    print('#'*40, f'폴드 {idx+1} / 폴드 {folds.n_splits}', '#'*40)
    
    # 훈련용 데이터, 검증용 데이터 설정
    X_train, y_train = X[train_idx], y[train_idx] # 훈련용 데이터
    X_valid, y_valid = X[valid_idx], y[valid_idx] # 검증용 데이터

    # lgb 데이터세트 생성
    dtrain = lgb.Dataset(X_train, y_train) # lgb 훈련 데이터세트
    dvalid = lgb.Dataset(X_valid, y_valid) # lgb 검증 데이터세트
                          
    # LightGBM 모델 훈련
    lgb_model = lgb.train(params=max_params, # 최적 하이퍼 파라미터
                          train_set=dtrain, # 훈련 데이터
                          num_boost_round=1500, # 부스팅 반복 횟수
                          valid_sets=dvalid, # 모델 성능 평가용 검증 데이터
                          feval=gini, # 검증용 평가지표
                          early_stopping_rounds=150, # 조기종료 조건
                          verbose_eval=100) # 100번째마다 점수 출력
    
    # 모델 성능이 가장 좋을 때의 부스팅 반복 횟수 저장
    best_iter = lgb_model.best_iteration
    # 테스트 데이터를 활용해 OOF 예측
    oof_test_preds += lgb_model.predict(X_test, 
                                    num_iteration=best_iter)/folds.n_splits
    
    # 모델 성능 평가를 위한 검증 데이터 타깃 값 예측 
    oof_val_preds[valid_idx] += lgb_model.predict(X_valid, 
                                                  num_iteration=best_iter)
    
    # 검증 데이터 예측확률에 대한 정규화 지니계수
    gini_score = eval_gini(y_valid, oof_val_preds[valid_idx])
    print(f'폴드 {idx+1} 지니계수: {gini_score}\n')

######################################## 폴드 1 / 폴드 5 ########################################
[LightGBM] [Info] Number of positive: 17355, number of negative: 458814
[LightGBM] [Info] Total Bins 1554
[LightGBM] [Info] Number of data points in the train set: 476169, number of used features: 216
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.036447 -> initscore=-3.274764
[LightGBM] [Info] Start training from score -3.274764
Training until validation scores don't improve for 150 rounds
[100]	valid_0's binary_logloss: 0.153315	valid_0's gini: 0.270034
[200]	valid_0's binary_logloss: 0.152339	valid_0's gini: 0.280485
[300]	valid_0's binary_logloss: 0.151922	valid_0's gini: 0.286543
[400]	valid_0's binary_logloss: 0.15169	valid_0's gini: 0.291089
[500]	valid_0's binary_logloss: 0.151544	valid_0's gini: 0.294535
[600]	valid_0's binary_logloss: 0.151467	valid_0's gini: 0.296222
[700]	valid_0's binary_logloss: 0.151405	valid_0's gini: 0.297658
[800]	valid_0's binary_logloss: 0.151368	valid_0

In [24]:
print('OOF 검증 데이터 지니계수:', eval_gini(y, oof_val_preds))

OOF 검증 데이터 지니계수: 0.2893420267793303


## 5.4.4 결과 제출

In [25]:
submission['target'] = oof_test_preds
submission.to_csv('submission.csv')